## Part 1: Scraping Data

In [276]:
# Author - Ashwin Srinivas
# Date - 5/1/2019

# Import requests package to request webpage information
import requests
# Import BeautifulSoup to parse HTML
from bs4 import BeautifulSoup
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from tkinter import Tk
from tkinter.filedialog import askopenfilename

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ashwi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [198]:
# open thr page with 100 recipes
r1 = requests.get('https://www.tasteofhome.com/collection/our-100-highest-rated-recipes-ever/')

In [199]:
# Parse HTML document
soup1 = BeautifulSoup(r1.text, 'html.parser')

In [200]:
# store the list of recipes in list recipe_links
recipe_links =[]

for i in soup1.find_all("span", {"class" : "listicle-page__cta-button"}):
    # this will not output the first first link as we are only looking for <span>
    recipe_links.append(i)

In [201]:
#append the first link (i.e. the <p> element) 
recipe_links.append(soup1.find("p", {"class" : "listicle-page__cta-button"})) 

In [202]:
# check if all 100 links are copied
len(recipe_links)

100

In [216]:
# this will contain the whole <span> tag. We need to extract the contents of the a-tag (i.e. href)
print(recipe_links[0:10])

[<span class="listicle-page__cta-button"><a class="no-smoothstate" href="https://www.tasteofhome.com/recipes/mini-caramel-rolls/">Get Recipe</a></span>, <span class="listicle-page__cta-button"><a class="no-smoothstate" href="https://www.tasteofhome.com/recipes/three-bean-baked-beans/">Get Recipe</a></span>, <span class="listicle-page__cta-button"><a class="no-smoothstate" href="https://www.tasteofhome.com/recipes/pepper-poppers/">Get Recipe</a></span>, <span class="listicle-page__cta-button"><a class="no-smoothstate" href="https://www.tasteofhome.com/recipes/sour-cream-chip-muffins/">Get Recipe</a></span>, <span class="listicle-page__cta-button"><a class="no-smoothstate" href="https://www.tasteofhome.com/recipes/onion-beef-au-jus/">Get Recipe</a></span>, <span class="listicle-page__cta-button"><a class="no-smoothstate" href="https://www.tasteofhome.com/recipes/tasty-italian-vegetable-soup/">Get Recipe</a></span>, <span class="listicle-page__cta-button"><a class="no-smoothstate" href="h

In [224]:
# store the final list of URLs in final_recipe_url1 (contents of href)
final_recipe_url1 = []
for i in recipe_links:
    a=i.find("a",href=True)
    final_recipe_url1.append(a['href'])
    
final_recipe_url1[0:10]

['https://www.tasteofhome.com/recipes/mini-caramel-rolls/',
 'https://www.tasteofhome.com/recipes/three-bean-baked-beans/',
 'https://www.tasteofhome.com/recipes/pepper-poppers/',
 'https://www.tasteofhome.com/recipes/sour-cream-chip-muffins/',
 'https://www.tasteofhome.com/recipes/onion-beef-au-jus/',
 'https://www.tasteofhome.com/recipes/tasty-italian-vegetable-soup/',
 'https://www.tasteofhome.com/recipes/apple-dumpling-bake/',
 'https://www.tasteofhome.com/recipes/crown-roast-of-pork-with-mushroom-dressing/',
 'https://www.tasteofhome.com/recipes/tasty-lentil-tacos/',
 'https://www.tasteofhome.com/recipes/garlic-roasted-brussels-sprouts-with-mustard-sauce/']

In [205]:
len(final_recipe_url1)

100

In [232]:
df_raw_data = pd.DataFrame(columns=['URL', 'RecipeName', 'Ingredients']) # new DF to store the recipes
i=0
for url in final_recipe_url1: # for each URL in the list, get the Title and ingredients and store it in a dataframe
    print("URL: ",url)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title = soup.find("h1", {"class" : "recipe-title"}).getText() # stores the title
    print("Title: ",title)
    list_of_ingredients = soup.find("div", {"class" : "recipe-ingredients"}).findAll('li') # list of ingredients
    print(list_of_ingredients)
    
    #iterate throught the list of ingredients to store each ingredient in a dataframe
    for ingredient in list_of_ingredients: 
        df_raw_data.loc[i] = [url] + [title] + [ingredient.text] # populating the data frame
        i+=1
    

URL:  https://www.tasteofhome.com/recipes/mini-caramel-rolls/
Title:  Mini Caramel Rolls
[<li>1/3 cup packed brown sugar</li>, <li>1/3 cup butter, cubed</li>, <li>2 tablespoons light corn syrup</li>, <li>1-1/2 teaspoons 2% milk</li>, <li>1 tube (8 ounces) refrigerated crescent rolls</li>, <li>2 teaspoons sugar</li>, <li>1/2 teaspoon ground cinnamon</li>]
URL:  https://www.tasteofhome.com/recipes/three-bean-baked-beans/
Title:  Three-Bean Baked Beans
[<li>1/2 pound ground beef</li>, <li>5 bacon strips, diced</li>, <li>1/2 cup chopped onion</li>, <li>1/3 cup packed brown sugar</li>, <li>1/4 cup sugar</li>, <li>1/4 cup ketchup</li>, <li>1/4 cup barbecue sauce</li>, <li>2 tablespoons molasses</li>, <li>2 tablespoons prepared mustard</li>, <li>1/2 teaspoon chili powder</li>, <li>1/2 teaspoon salt</li>, <li>2 cans (16 ounces each) pork and beans, undrained</li>, <li>1 can (16 ounces) butter beans, rinsed and drained</li>, <li>1 can (16 ounces) kidney beans, rinsed and drained</li>]
URL:  htt

Title:  Caribbean-Spiced Pork Tenderloin with Peach Salsa
[<li>3/4 cup chopped peeled fresh peaches</li>, <li>1 small sweet red pepper, chopped</li>, <li>1 jalapeno pepper, seeded and chopped</li>, <li>2 tablespoons finely chopped red onion</li>, <li>2 tablespoons minced fresh cilantro</li>, <li>1 tablespoon lime juice</li>, <li>1 garlic clove, minced</li>, <li>1/8 teaspoon salt</li>, <li>1/8 teaspoon pepper</li>, <li>2 tablespoons olive oil</li>, <li>1 tablespoon brown sugar</li>, <li>1 tablespoon Caribbean jerk seasoning</li>, <li>1 teaspoon dried thyme</li>, <li>1 teaspoon dried rosemary, crushed</li>, <li>1/2 teaspoon seasoned salt</li>, <li>1 pork tenderloin (1 pound)</li>]
URL:  https://www.tasteofhome.com/recipes/mushroom-wild-rice-soup/
Title:  Mushroom & Wild Rice Soup
[<li>2-1/2 cups water</li>, <li>1 ounce dried porcini mushrooms</li>, <li>1 ounce dried shiitake mushrooms</li>, <li>3 tablespoons butter</li>, <li>1 small onion, finely chopped</li>, <li>1/2 pound sliced fresh 

Title:  Rhubarb Mandarin Crisp
[<li>6 cups chopped fresh or frozen rhubarb</li>, <li>1-1/2 cups sugar</li>, <li>5 tablespoons quick-cooking tapioca</li>, <li>1 can (11 ounces) mandarin oranges, drained</li>, <li>1 cup packed brown sugar</li>, <li>1 cup quick-cooking oats</li>, <li>1/2 cup all-purpose flour</li>, <li>1/2 teaspoon salt</li>, <li>1/2 cup cold butter, cubed</li>, <li>Ice cream, optional</li>]
URL:  https://www.tasteofhome.com/recipes/grilled-chicken-salad-with-blueberry-vinaigrette/
Title:  Grilled Chicken Salad with Blueberry Vinaigrette
[<li>2 boneless skinless chicken breast halves (6 ounces each)</li>, <li>1 tablespoon olive oil</li>, <li>1 garlic clove, minced</li>, <li>1/4 teaspoon salt</li>, <li>1/4 teaspoon pepper</li>, <li><b class="sIngredient">VINAIGRETTE:</b></li>, <li>1/4 cup olive oil</li>, <li>1/4 cup blueberry preserves</li>, <li>2 tablespoons balsamic vinegar</li>, <li>2 tablespoons maple syrup</li>, <li>1/4 teaspoon ground mustard</li>, <li>1/8 teaspoon s

Title:  Standing Rib Roast
[<li>3 teaspoons lemon-pepper seasoning</li>, <li>3 teaspoons paprika</li>, <li>1-1/2 teaspoons garlic salt</li>, <li>1 teaspoon dried rosemary, crushed</li>, <li>1/2 teaspoon cayenne pepper</li>, <li>1 bone-in beef rib roast (6 to 7 pounds)</li>, <li>2 cups beef stock</li>]
URL:  https://www.tasteofhome.com/recipes/blueberry-crunch-breakfast-bake/
Title:  Blueberry Crunch Breakfast Bake
[<li>1 loaf (16 ounces) day-old French bread, cut into 1-inch slices</li>, <li>8 large <strong>Eggland's Best eggs</strong> <div class="recipe-featured-ingredient-expanded branded-ingredient" data-ad="branded-ingredient" data-targeting='{"campaign_id":"Eg23","branded":"large-eggs"}' id="branded-ingredient-111">
</div></li>, <li>1 cup half-and-half cream</li>, <li>1/2 teaspoon vanilla extract</li>, <li>1 cup old-fashioned oats</li>, <li>1 cup packed brown sugar</li>, <li>1/4 cup all-purpose flour</li>, <li>1/2 cup cold butter</li>, <li>2 cups fresh or frozen blueberries</li>, 

Title:  Fire-and-Ice Pickles
[<li>2 jars (32 ounces each) dill pickle slices or spears</li>, <li>4 cups sugar</li>, <li>1 tablespoon hot pepper sauce</li>, <li>1/2 teaspoon crushed red pepper flakes</li>, <li>3 garlic cloves, peeled</li>]
URL:  https://www.tasteofhome.com/recipes/chicken-sausage-gnocchi-skillet/
Title:  Chicken Sausage & Gnocchi Skillet
[<li>1 package (16 ounces) potato gnocchi</li>, <li>1 tablespoon butter</li>, <li>1 tablespoon olive oil</li>, <li>2 fully cooked Italian chicken sausage links (3 ounces each), sliced</li>, <li>1/2 pound sliced baby portobello mushrooms</li>, <li>1 medium onion, finely chopped</li>, <li>1 pound fresh asparagus, trimmed and cut into 1/2-inch pieces</li>, <li>2 garlic cloves, minced</li>, <li>2 tablespoons white wine or  chicken broth</li>, <li>2 ounces herbed fresh goat cheese</li>, <li>2 tablespoons minced fresh basil or 2 teaspoons dried basil</li>, <li>1 tablespoon lemon juice</li>, <li>1/4 teaspoon salt</li>, <li>1/8 teaspoon pepper<

Title:  Eggsquisite Breakfast Casserole
[<li>1 pound sliced bacon, diced</li>, <li>2 packages (4-1/2 ounces each) sliced dried beef, cut into thin strips</li>, <li>1 can (4 ounces) sliced mushrooms</li>, <li>1/2 cup all-purpose flour</li>, <li>1/8 teaspoon pepper</li>, <li>4 cups whole milk</li>, <li>16 large <strong>Eggland's Best eggs</strong> <div class="recipe-featured-ingredient-expanded branded-ingredient" data-ad="branded-ingredient" data-targeting='{"campaign_id":"Eg23","branded":"large-eggs"}' id="branded-ingredient-408">
</div></li>, <li>1 cup evaporated milk</li>, <li>1/4 teaspoon salt</li>, <li>1/4 cup butter, cubed</li>, <li>Chopped fresh parsley, optional</li>]
URL:  https://www.tasteofhome.com/recipes/gingerbread-snowflakes/
Title:  Gingerbread Snowflakes
[<li>1 cup butter, softened</li>, <li>1 cup sugar</li>, <li>1 cup molasses</li>, <li>1/4 cup water</li>, <li>5 cups all-purpose flour</li>, <li>2-1/2 teaspoons ground ginger</li>, <li>1-1/2 teaspoons baking soda</li>, <

Title:  Skillet Ham & Rice
[<li>1 teaspoon olive oil</li>, <li>1 medium onion, chopped</li>, <li>1 cup sliced fresh mushrooms</li>, <li>1 cup cubed fully cooked ham</li>, <li>1/8 teaspoon pepper</li>, <li>1/2 cup reduced-sodium chicken broth</li>, <li>1/4 cup water</li>, <li>3/4 cup uncooked instant rice</li>, <li>2 green onions, sliced</li>, <li>1/4 cup shredded Parmesan cheese</li>]
URL:  https://www.tasteofhome.com/recipes/cranberry-sauce/
Title:  Cranberry Sauce
[<li>1 package (12 ounces) fresh or frozen cranberries</li>, <li>1 cup sugar</li>, <li>1 cup cranberry-raspberry juice</li>, <li>1 tablespoon lemon juice</li>]
URL:  https://www.tasteofhome.com/recipes/rich-chicken-alfredo-pizza/
Title:  Rich Chicken Alfredo Pizza
[<li>2-1/2 teaspoons butter</li>, <li>1 garlic clove, minced</li>, <li>1-1/2 cups heavy whipping cream</li>, <li>3 tablespoons grated Parmesan cheese</li>, <li>1/2 teaspoon salt</li>, <li>1/4 teaspoon pepper</li>, <li>1 tablespoon minced fresh parsley</li>, <li>1 

In [234]:
# check if it has populated correctly
df_raw_data.head(30)

,URL,RecipeName,Ingredients
0,https://www.tasteofhome.com/recipes/mini-caram...,Mini Caramel Rolls,1/3 cup packed brown sugar
1,https://www.tasteofhome.com/recipes/mini-caram...,Mini Caramel Rolls,"1/3 cup butter, cubed"
2,https://www.tasteofhome.com/recipes/mini-caram...,Mini Caramel Rolls,2 tablespoons light corn syrup
3,https://www.tasteofhome.com/recipes/mini-caram...,Mini Caramel Rolls,1-1/2 teaspoons 2% milk
4,https://www.tasteofhome.com/recipes/mini-caram...,Mini Caramel Rolls,1 tube (8 ounces) refrigerated crescent rolls
5,https://www.tasteofhome.com/recipes/mini-caram...,Mini Caramel Rolls,2 teaspoons sugar
6,https://www.tasteofhome.com/recipes/mini-caram...,Mini Caramel Rolls,1/2 teaspoon ground cinnamon
7,https://www.tasteofhome.com/recipes/three-bean...,Three-Bean Baked Beans,1/2 pound ground beef
8,https://www.tasteofhome.com/recipes/three-bean...,Three-Bean Baked Beans,"5 bacon strips, diced"
9,https://www.tasteofhome.com/recipes/three-bean...,Three-Bean Baked Beans,1/2 cup chopped onion


In [236]:
# store in a csv file
df_raw_data.to_csv("ashwin_raw_data.csv")

In [294]:
Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
df_raw_data=pd.read_csv(filename)

## Part 2: Cleaning Data

In [295]:
df_raw_data['Ingredients'][0]

'1/3 cup packed brown sugar'

In [296]:
# If character is not a alphabet or space, replace with blank
df_raw_data['Ingredients'].replace(regex=True,inplace=True,to_replace=r'[^a-zA-Z ]',value=r'')

# remove leading and trailing whitespace
df_raw_data['Ingredients'] = df_raw_data['Ingredients'].str.strip().str.lower() 

df_raw_data['Ingredients'][10:30]

10                               cup packed brown sugar
11                                            cup sugar
12                                          cup ketchup
13                                   cup barbecue sauce
14                                 tablespoons molasses
15                         tablespoons prepared mustard
16                                teaspoon chili powder
17                                        teaspoon salt
18           cans  ounces each pork and beans undrained
19          can  ounces butter beans rinsed and drained
20          can  ounces kidney beans rinsed and drained
21                package  ounces cream cheese softened
22                    cup shredded sharp cheddar cheese
23                    cup shredded monterey jack cheese
24                     bacon strips cooked and crumbled
25                                        teaspoon salt
26                               teaspoon garlic powder
27                                teaspoon chili

In [297]:
df_raw_data1 = pd.DataFrame()
df_raw_data1 = df_raw_data # Save this in another data frame


In [301]:
df_raw_data['Ingredients'][0:50]

0                                cup packed brown sugar
1                                      cup butter cubed
2                          tablespoons light corn syrup
3                                       teaspoons  milk
4              tube  ounces refrigerated crescent rolls
5                                       teaspoons sugar
6                              teaspoon ground cinnamon
7                                     pound ground beef
8                                    bacon strips diced
9                                     cup chopped onion
10                               cup packed brown sugar
11                                            cup sugar
12                                          cup ketchup
13                                   cup barbecue sauce
14                                 tablespoons molasses
15                         tablespoons prepared mustard
16                                teaspoon chili powder
17                                        teaspo

In [300]:
# store in a csv file
df_raw_data.to_csv("ashwin_raw_data1.csv")

In [306]:

for i in range(0,len(df_raw_data['Ingredients'])):
    df_raw_data['Ingredients'][i].split()

df_raw_data['Ingredients'][0]

'cup packed brown sugar'

In [311]:
df_raw_data['IngredientsCleaned'] = df_raw_data['Ingredients'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [308]:
df_raw_data['IngredientsNoStopWords'][0:30]

0                             cup packed brown sugar
1                                   cup butter cubed
2                       tablespoons light corn syrup
3                                     teaspoons milk
4            tube ounces refrigerated crescent rolls
5                                    teaspoons sugar
6                           teaspoon ground cinnamon
7                                  pound ground beef
8                                 bacon strips diced
9                                  cup chopped onion
10                            cup packed brown sugar
11                                         cup sugar
12                                       cup ketchup
13                                cup barbecue sauce
14                              tablespoons molasses
15                      tablespoons prepared mustard
16                             teaspoon chili powder
17                                     teaspoon salt
18                  cans ounces pork beans und

In [314]:
df_raw_data['Ingredients'][80:100]

80                      medium onion finely chopped
81                              garlic clove minced
82                         cup dried lentils rinsed
83                          tablespoon chili powder
84                           teaspoons ground cumin
85                           teaspoon dried oregano
86    cups vegetable or reducedsodium chicken broth
87                                        cup salsa
88                                      taco shells
89                            cups shredded lettuce
90                       cup chopped fresh tomatoes
91          cups shredded reducedfat cheddar cheese
92                   tablespoons fatfree sour cream
93             pounds fresh brussels sprouts halved
94                            tablespoons olive oil
95                             garlic cloves minced
96                         cup heavy whipping cream
97                        tablespoons dijon mustard
98                            teaspoon white pepper
99          

In [ ]:
# Create list of extra words
extras = ['can','teaspoon','dry','tablespoon','and','cup','pound','ounce','rinsed','package','strips','drained','crumbled','grated','medium','small','large','to','pounds','about','whole','fresh','preserved','seasoned','slices','into','cut','tubes','softned','dash']

In [333]:
# Create new column with blank strings
df_raw_data['IngredientsCleaned'] = "" 

#populate IngredientsCleaned column if word is not in "extras"
for i in range(0,len(df_raw_data['IngredientsNoStopWords'])):
    x = df_raw_data['IngredientsNoStopWords'][i].split() # split the line into list of words
    #print(x)
    for j in x:
        if j in extras:
            #print("extra",j)
            continue
        else:
            df_raw_data['IngredientsCleaned'][i]=''.join(j)
            #print(df_raw_data['IngredientsCleaned'][i])

['cup', 'packed', 'brown', 'sugar']
extra cup
extra packed
brown
sugar
['cup', 'butter', 'cubed']
extra cup
butter
extra cubed
['tablespoons', 'light', 'corn', 'syrup']
extra tablespoons
light


C:\Users\ashwi\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


corn
syrup
['teaspoons', 'milk']
extra teaspoons
milk
['tube', 'ounces', 'refrigerated', 'crescent', 'rolls']
tube
extra ounces
refrigerated
crescent
rolls
['teaspoons', 'sugar']
extra teaspoons
sugar
['teaspoon', 'ground', 'cinnamon']
teaspoon
ground
cinnamon
['pound', 'ground', 'beef']
pound
ground
beef
['bacon', 'strips', 'diced']
bacon
strips
diced
['cup', 'chopped', 'onion']
extra cup
chopped
onion
['cup', 'packed', 'brown', 'sugar']
extra cup
extra packed
brown
sugar
['cup', 'sugar']
extra cup
sugar
['cup', 'ketchup']
extra cup
ketchup
['cup', 'barbecue', 'sauce']
extra cup
barbecue
sauce
['tablespoons', 'molasses']
extra tablespoons
molasses
['tablespoons', 'prepared', 'mustard']
extra tablespoons
prepared
mustard
['teaspoon', 'chili', 'powder']
teaspoon
chili
powder
['teaspoon', 'salt']
teaspoon
salt
['cans', 'ounces', 'pork', 'beans', 'undrained']
cans
extra ounces
pork
beans
undrained
['ounces', 'butter', 'beans', 'rinsed', 'drained']
extra ounces
butter
beans
rinsed
drained


uncooked
bratwurst
links
['pounds', 'small', 'red', 'potatoes', 'cut', 'wedges']
pounds
small
red
potatoes
cut
wedges
['pound', 'baby', 'carrots']
pound
baby
carrots
['large', 'red', 'onion', 'sliced', 'separated', 'rings']
large
red
onion
sliced
separated
rings
['jars', 'ounces', 'whole', 'mushrooms', 'drained']
jars
extra ounces
whole
mushrooms
drained
['cup', 'butter', 'cubed']
extra cup
butter
extra cubed
['envelope', 'onion', 'soup', 'mix']
envelope
onion
soup
mix
['tablespoons', 'soy', 'sauce']
extra tablespoons
soy
sauce
['teaspoon', 'pepper']
teaspoon
pepper
['packages', 'ounce', 'active', 'dry', 'yeast']
packages
ounce
active
dry
yeast
['cups', 'warm', 'water']
cups
warm
water
['cup', 'sugar']
extra cup
sugar
['tablespoon', 'canola', 'oil']
tablespoon
canola
oil
['cup', 'whole', 'wheat', 'flour']
extra cup
whole
wheat
flour
['large', 'eggs']
large
eggs
['tablespoon', 'water']
tablespoon
water
['tablespoon', 'salt']
tablespoon
salt
['cups', 'allpurpose', 'flour']
cups
allpurpos

extract
['topping']
topping
['cup', 'packed', 'brown', 'sugar']
extra cup
extra packed
brown
sugar
['cup', 'sweetened', 'shredded', 'coconut']
extra cup
sweetened
shredded
coconut
['cup', 'chopped', 'pecans']
extra cup
chopped
pecans
['cup', 'reducedfat', 'butter', 'melted']
extra cup
reducedfat
butter
melted
['cup', 'butter', 'softened']
extra cup
butter
softened
['cup', 'confectioners', 'sugar']
extra cup
confectioners
sugar
['teaspoons', 'vanilla', 'extract']
extra teaspoons
vanilla
extract
['cups', 'allpurpose', 'flour']
cups
allpurpose
flour
['teaspoon', 'salt']
teaspoon
salt
['filling']
filling
['ounces', 'cream', 'cheese', 'softened']
extra ounces
cream
cheese
softened
['cup', 'confectioners', 'sugar']
extra cup
confectioners
sugar
['tablespoons', 'allpurpose', 'flour']
extra tablespoons
allpurpose
flour
['teaspoon', 'vanilla', 'extract']
teaspoon
vanilla
extract
['cup', 'finely', 'chopped', 'pecans']
extra cup
finely
chopped
pecans
['cup', 'sweetened', 'shredded', 'coconut']
ex

...

roast
pounds
['cups', 'beef', 'stock']
cups
beef
stock
['loaf', 'ounces', 'dayold', 'french', 'bread', 'cut', 'inch', 'slices']
loaf
extra ounces
dayold
french
bread
cut
inch
slices
['large', 'egglands', 'best', 'eggs']
large
egglands
best
eggs
['cup', 'halfandhalf', 'cream']
extra cup
halfandhalf
cream
['teaspoon', 'vanilla', 'extract']
teaspoon
vanilla
extract
['cup', 'oldfashioned', 'oats']
extra cup
oldfashioned
oats
['cup', 'packed', 'brown', 'sugar']
extra cup
extra packed
brown
sugar
['cup', 'allpurpose', 'flour']
extra cup
allpurpose
flour
['cup', 'cold', 'butter']
extra cup
cold
butter
['cups', 'fresh', 'frozen', 'blueberries']
cups
fresh
frozen
blueberries
['cup', 'chopped', 'walnuts']
extra cup
chopped
walnuts
['cup', 'seasoned', 'bread', 'crumbs', 'divided']
extra cup
seasoned
bread
crumbs
divided
['cup', 'finely', 'chopped', 'onion']
extra cup
finely
chopped
onion
['teaspoons', 'lowsodium', 'worcestershire', 'sauce']
extra teaspoons
lowsodium
worcestershire
sauce
['teaspoo

melted
['gallon', 'vanilla', 'ice', 'cream', 'softened']
gallon
vanilla
ice
cream
softened
['jar', 'ounces', 'hot', 'fudge', 'ice', 'cream', 'topping', 'warmed']
jar
extra ounces
hot
fudge
ice
cream
topping
warmed
['carton', 'ounces', 'frozen', 'whipped', 'topping', 'thawed']
carton
extra ounces
frozen
whipped
topping
thawed
['prebaked', 'inch', 'pizza', 'crust']
prebaked
inch
pizza
crust
['cup', 'miracle', 'whip']
extra cup
miracle
whip
['teaspoons', 'dried', 'basil']
extra teaspoons
dried
basil
['teaspoon', 'garlic', 'powder']
teaspoon
garlic
powder
['teaspoon', 'onion', 'powder']
teaspoon
onion
powder
['bacon', 'strips', 'cooked', 'crumbled']
bacon
strips
cooked
crumbled
['cup', 'shredded', 'cheddar', 'cheese']
extra cup
shredded
cheddar
cheese
['cup', 'shredded', 'partskim', 'mozzarella', 'cheese']
extra cup
shredded
partskim
mozzarella
cheese
['cups', 'shredded', 'lettuce']
cups
shredded
lettuce
['medium', 'tomatoes', 'thinly', 'sliced']
medium
tomatoes
thinly
sliced
['cups', 'unc

barbecued
shredded
pork
['ounces', 'pineapple', 'chunks', 'drained']
extra ounces
pineapple
chunks
drained
['cup', 'chopped', 'red', 'onion']
extra cup
chopped
red
onion
['cups', 'shredded', 'partskim', 'mozzarella', 'cheese']
cups
shredded
partskim
mozzarella
cheese
['cups', 'diced', 'cooked', 'turkey']
cups
diced
cooked
turkey
['cups', 'cooked', 'long', 'grain', 'rice']
cups
cooked
long
grain
rice
['ounces', 'condensed', 'cream', 'chicken', 'soup', 'undiluted']
extra ounces
condensed
cream
chicken
soup
undiluted
['teaspoon', 'pepper']
teaspoon
pepper
['cups', 'chicken', 'broth', 'divided']
cups
chicken
broth
divided
['tablespoons', 'cornstarch']
extra tablespoons
cornstarch
['cup', 'lemon', 'juice']
extra cup
lemon
juice
['cup', 'minced', 'fresh', 'cilantro']
extra cup
minced
fresh
cilantro
['package', 'devils', 'food', 'cake', 'mix', 'regular', 'size']
package
devils
food
cake
mix
regular
size
['package', 'ounces', 'cream', 'cheese', 'softened']
package
extra ounces
cream
cheese
sof

juice
['tablespoons', 'olive', 'oil']
extra tablespoons
olive
oil
['teaspoon', 'salt']
teaspoon
salt
['teaspoon', 'pepper']
teaspoon
pepper
['pounds', 'potatoes', 'medium', 'sliced', 'inch', 'thick']
pounds
potatoes
medium
sliced
inch
thick
['cooking', 'spray']
cooking
spray
['teaspoons', 'sea', 'salt']
extra teaspoons
sea
salt
['teaspoon', 'coarsely', 'ground', 'pepper']
teaspoon
coarsely
ground
pepper
['green', 'onions', 'thinly', 'sliced', 'cup']
green
onions
thinly
sliced
extra cup
['tablespoons', 'minced', 'fresh', 'parsley']
extra tablespoons
minced
fresh
parsley
['tablespoons', 'minced', 'fresh', 'tarragon']
extra tablespoons
minced
fresh
tarragon
['olive', 'oil', 'optional']
olive
oil
optional
['medium', 'cucumber', 'peeled']
medium
cucumber
peeled
['cup', 'reducedfat', 'sour', 'cream']
extra cup
reducedfat
sour
cream
['cup', 'finely', 'chopped', 'onion']
extra cup
finely
chopped
onion
['teaspoons', 'dill', 'weed']
extra teaspoons
dill
weed
['teaspoons', 'lemon', 'juice']
extra

cooked
chicken
breast
['cup', 'buffalo', 'wing', 'sauce']
extra cup
buffalo
wing
sauce
['cup', 'ranch', 'blue', 'cheese', 'salad', 'dressing']
extra cup
ranch
blue
cheese
salad
dressing
['cups', 'shredded', 'colbymonterey', 'jack', 'cheese']
cups
shredded
colbymonterey
jack
cheese
['french', 'bread', 'baguette', 'slices', 'celery', 'ribs', 'tortilla', 'chips']
french
bread
baguette
slices
celery
ribs
tortilla
chips


In [336]:
df_raw_data['IngredientsCleaned'][0:30]

0         sugar
1        butter
2         syrup
3          milk
4         rolls
5         sugar
6      cinnamon
7          beef
8         diced
9         onion
10        sugar
11        sugar
12      ketchup
13        sauce
14     molasses
15      mustard
16       powder
17         salt
18    undrained
19      drained
20      drained
21     softened
22       cheese
23       cheese
24     crumbled
25         salt
26       powder
27       powder
28       seeded
29       crumbs
Name: IngredientsCleaned, dtype: object

In [337]:
# store in a csv file
df_raw_data.to_csv("ashwin_raw_data2.csv")

## Part 3: Calculating the top 10 ingredients

In [338]:
df_raw_data['IngredientCount'] = 1 # add new column with 1 to count each ingredient

In [340]:
# group by each ingredient and sum the count

df_raw_data2 = pd.DataFrame(df_raw_data.groupby('IngredientsCleaned').count().sort_values(by='IngredientCount', ascending=False))

In [343]:
# filter only necessary columns
df_raw_data2 = df_raw_data2.filter(['IngredientsCleaned','IngredientCount'], axis=1) 

In [347]:
# Calculate the proportion

df_raw_data2['Proportion'] = df_raw_data2['IngredientCount']/100

### Top 10 ingredients

In [351]:
# get the top 10 ingeredients
df_raw_data2.reset_index().head(10)

,IngredientsCleaned,IngredientCount,Proportion
0,salt,56,0.56
1,sugar,52,0.52
2,cheese,44,0.44
3,pepper,42,0.42
4,oil,31,0.31
5,chopped,28,0.28
6,sauce,27,0.27
7,minced,24,0.24
8,sliced,23,0.23
9,flour,22,0.22


In [352]:
# store in a csv file
df_raw_data2.head(10).to_csv("ashwin_results.csv")